In [22]:
import sys
!{sys.executable} -m pip install pyarrow

Bloco 1: Importação de Bibliotecas e Carregamento dos Dados Limpos

In [23]:
import pandas as pd
import numpy as np

Bloco 2: Carregamento dos Dados Processados


In [24]:
caminho_dados_limpos = '../data/processed/dados_limpos.parquet'
df = pd.read_parquet(caminho_dados_limpos)

print("Dados limpos carregados com sucesso!")
display(df.head())

Dados limpos carregados com sucesso!


,nk_ota_localizer_id,fk_contact,place_origin_departure,place_destination_departure,place_origin_return,place_destination_return,fk_departure_ota_bus_company,fk_return_ota_bus_company,gmv_success,total_tickets_quantity_success,datetime_purchase
0,bc02d5245bec63b30ff1102fa273fc03f58bc9cc3f674e...,a7218ff4ee7d37d48d2b4391b955627cb089870b934912...,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,50e9a8665b62c8d68bccc77c7c92431a1aa26ccbd38ed4...,0,0,8527a891e224136950ff32ca212b45bc93f69fbb801c3b...,1,89.09,1,2018-12-26 15:33:35
1,5432f12612dd5d749b3be880e779989cf63b5efa4bcc4e...,37228485e0dc83d84d1bcd1bef3dc632301bf6cb22c8b5...,10e4e7caf8b078429bb1c80b1a10118ac6f963eff098fd...,e6d41d208672a4e50b86d959f4a6254975e6fb9b088116...,0,0,36ebe205bcdfc499a25e6923f4450fa8d48196ceb4fa0c...,1,155.97,1,2018-12-05 15:07:57
2,fb3caed9b2f1b6016d45ccddb19095476e61a2c85faa8e...,3467ec081e2421e72c96e7203b929d21927fd00b6b5f28...,7688b6ef52555962d008fff894223582c484517cea7da4...,8c1f1046219ddd216a023f792356ddf127fce372a72ec9...,0,0,ec2e990b934dde55cb87300629cedfc21b15cd28bbcf77...,1,121.99,1,2018-12-21 18:41:54
3,4dc44a6dd592b702feccb493d192210c86965aee684529...,ab3251a2be0f69713b8f97b0e9d1579e31551f4fd4facf...,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,d6acb3c1a79e57bcc03d976cb4d98f56edccd4cf426392...,0,0,5f9c4ab08cac7457e9111a30e4664920607ea2c115a143...,1,55.22,1,2018-12-06 14:01:38
4,aa34ed7fd0a6b405df2df1bf9f8d68e6df9b9a868a6181...,ceea0de820a6379f2c4215bddaec66c33994b304607e56...,7688b6ef52555962d008fff894223582c484517cea7da4...,23765fc69c4e3c0b10f5d15471dc2245e2a19af16b513f...,0,0,48449a14a4ff7d79bb7a1b6f3d488eba397c36ef25634c...,1,45.31,1,2021-02-23 20:08:25


Bloco 2.1: Carregar e Aplicar Aliases


In [25]:
import json

# Carregar os dicionários de mapeamento
with open('../data/processed/mapa_clientes_alias.json', 'r') as f:
    mapa_clientes = json.load(f)
with open('../data/processed/mapa_locais_alias.json', 'r') as f:
    mapa_locais = json.load(f)

print("Mapeamentos de alias carregados.")

# Aplicar os aliases para criar novas colunas legíveis
df['nome_cliente'] = df['fk_contact'].map(mapa_clientes)
df['origem_alias'] = df['place_origin_departure'].map(mapa_locais)
df['destino_alias'] = df['place_destination_departure'].map(mapa_locais)

# Criar a coluna de trecho já com os nomes legíveis
# df['trecho_alias'] = df['origem_alias'] + ' -> ' + df['destino_alias']
df['produto'] = df['origem_alias'] + ' -> ' + df['destino_alias']

print("Colunas de alias criadas com sucesso!")
# display(df[['nome_cliente', 'trecho_alias']].head())
display(df[['nome_cliente', 'produto']].head())

Mapeamentos de alias carregados.
Colunas de alias criadas com sucesso!


,nome_cliente,produto
0,Maria Clara da Paz,Mendes -> Nunes de Silveira
1,Valentina Aragão,Castro de Ramos -> Moreira do Oeste
2,Laís da Luz,Aparecida dos Dourados -> Vieira Paulista
3,Matheus da Cunha,Cavalcante -> Ribeiro
4,Dr. Davi Lucca da Conceição,Aparecida dos Dourados -> Mendonça de Pimenta


Bloco 3: Engenharia de Features Binárias (Comportamento de Viagem)


In [26]:
# Feature 1: tipo_viagem (Ida e Volta vs. Só Ida)
df['tipo_viagem'] = 'Ida e Volta'
df.loc[df['place_origin_return'] == '0', 'tipo_viagem'] = 'Só Ida'

# Feature 2: viaja_sozinho
df['viaja_sozinho'] = np.where(df['total_tickets_quantity_success'] == 1, 'Sim', 'Não')

# Feature 3: compra_fds (Sexta, Sábado e Domingo)
df['compra_fds'] = np.where(df['datetime_purchase'].dt.dayofweek >= 5, 'Sim', 'Não')

# Feature 4: compra_alta_temporada
meses_alta_temporada = [12, 1, 7] # Exemplo: Dezembro, Janeiro e Julho
df['compra_alta_temporada'] = np.where(df['datetime_purchase'].dt.month.isin(meses_alta_temporada), 'Sim', 'Não')

print("Novas features de comportamento criadas. Amostra:")
display(df[['fk_contact', 'tipo_viagem', 'viaja_sozinho', 'compra_fds', 'compra_alta_temporada']].head())

Novas features de comportamento criadas. Amostra:


,fk_contact,tipo_viagem,viaja_sozinho,compra_fds,compra_alta_temporada
0,a7218ff4ee7d37d48d2b4391b955627cb089870b934912...,Só Ida,Sim,Não,Sim
1,37228485e0dc83d84d1bcd1bef3dc632301bf6cb22c8b5...,Só Ida,Sim,Não,Sim
2,3467ec081e2421e72c96e7203b929d21927fd00b6b5f28...,Só Ida,Sim,Não,Sim
3,ab3251a2be0f69713b8f97b0e9d1579e31551f4fd4facf...,Só Ida,Sim,Não,Sim
4,ceea0de820a6379f2c4215bddaec66c33994b304607e56...,Só Ida,Sim,Não,Não


Bloco 4: Análise RFM (Agregação por Cliente)


In [27]:
# Usamos o dia seguinte à última compra registrada em todo o dataset para garantir que todas as recências sejam positivas.
snapshot_date = df['datetime_purchase'].max() + pd.Timedelta(days=1)

# Agrupando por cliente e calcular Recência, Frequência e Valor Monetário
df_rfm = df.groupby('fk_contact').agg({
    'datetime_purchase': lambda date: (snapshot_date - date.max()).days, # Recência
    'nk_ota_localizer_id': 'nunique', # Frequência (contagem de compras únicas)
    'gmv_success': 'sum' # Valor Monetário
})

# Renomeando as colunas para o padrão RFM, para maior clareza
df_rfm.rename(columns={'datetime_purchase': 'Recencia',
                       'nk_ota_localizer_id': 'Frequencia',
                       'gmv_success': 'ValorMonetario'}, inplace=True)

print("DataFrame RFM criado com sucesso:")
display(df_rfm.head())

DataFrame RFM criado com sucesso:


,Recencia,Frequencia,ValorMonetario
fk_contact,,,
0001018716456b2b34ca7a31f9b597974be6e1c9f6122a1bba5bb9c267a9e7fe,259,1,193.61
0004d0000beb6c47a5ae74d13af1d0c7e862101147492532fc32d80211558660,1502,1,81.82
00066779acf4b4cae18112860bbb556f0a0b609f5cbdfcefd5ebb574f7e7d600,1370,1,68.44
0007d4c7a554f2aa722aa7840d953395da4c019c36c51dda73c7678d635683c9,1919,1,467.28
000b40069834aaf5757373f9aac1e92968ccbcfff3838ccac21ab4a8eceb0df8,842,2,52.70


Bloco 5: Persistência dos Resultados


In [28]:

# A) Salvando o DataFrame de vendas, ALINHADO com o frontend
print("Alinhando e salvando o arquivo de vendas para o frontend...")
caminho_sales = '../data/redis/sales.parquet'

# Renomeando as colunas para bater com o que o app.py espera
df_para_app = df.rename(columns={
    'fk_contact': 'id_cliente',
    'datetime_purchase': 'data_venda', # Renomeia a coluna de data
    'gmv_success': 'valor_venda'    # Renomeia a coluna de valor
})

# Salvando o DataFrame com os nomes de coluna corretos
df_para_app.to_parquet(caminho_sales, index=False)
print(f"--> Arquivo 'sales.parquet' salvo em: '{caminho_sales}' com as colunas corretas.")


# Salvando o DataFrame RFM (para o notebook de modelagem)
print("\nSalvando o arquivo com as features RFM...")
caminho_rfm = '../data/curated/rfm_features.parquet'
df_rfm.to_parquet(caminho_rfm)
print(f"--> Arquivo RFM ('rfm_features.parquet') salvo em: '{caminho_rfm}'")

Alinhando e salvando o arquivo de vendas para o frontend...
--> Arquivo 'sales.parquet' salvo em: '../data/redis/sales.parquet' com as colunas corretas.

Salvando o arquivo com as features RFM...
--> Arquivo RFM ('rfm_features.parquet') salvo em: '../data/curated/rfm_features.parquet'
